In [ ]:
import pandas as pd
import numpy as np
import pickle
import random
from matplotlib import pyplot as plt
from monotonic_check import monotonicity_score, check_monotonicity
from critical_values_finder import find_critical_values, highest_and_lowest_indexes
from calibration_check import calibration_check

In [ ]:
#manter plots salver disco
#classe 

In [ ]:
with open('models/XGBoost/XGBoost.pkl', 'rb') as f:
    XGBoost = pickle.load(f)
with open('models/LogisticRegression/LogisticRegression.pkl', 'rb') as f:
    LR = pickle.load(f)
with open('models/RandomForest/RandomForest.pkl', 'rb') as f:
    RF = pickle.load(f)
models = [XGBoost, LR, RF]


In [ ]:
df = pd.read_csv('../datasets/fraud_new.csv')
example = df.sample(1)
example = example.drop('isFraud', axis = 1)

In [ ]:
for c in df.columns:
    if c != 'isFraud':
        minV = df[c].min()
        maxV = df[c].max()
        for model in models:
            find_critical_values(model, example, c, minV, maxV, )

In [ ]:
for c in df.columns:
    if c != 'isFraud':
        minV = df[c].min()
        maxV = df[c].max()
        for model in models:
            check_monotonicity(c,minV,maxV,example, model, 150, plot_graph=True)

In [ ]:
import math
v3= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,14,15,16,17,18,17,18,19,20,19,20,21,22,23,24,25,26,25,26,27,28,29,30,31,35,37,40,43,50,60]
plt.plot([x for x in range(len(v3))], v3)
plt.show()
print(monotonicity_score(v3))
v = [1,2,3,4,5,6,7,8,7,6,5,4,3,2,1,2,3,4,5,6,7,8,9] 
plt.plot([x for x in range(len(v))], v)
plt.show()
print(monotonicity_score(v))
v2= [1,2,3,2,1,3,4,5,6,5,4,6,7,8,9,8,10]
plt.plot([x for x in range(len(v2))], v2)
plt.show()
print(monotonicity_score(v2))


In [ ]:
for model in models:
    calibration_check("isFraud", model, df, True)